In [2]:
!pip install pymupdf #Required to run the code below


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 39.4 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#File Location and name

file_name = '1603.09631.pdf'
file_base = file_name.replace('.pdf', '')
pdf_file_path = '/content/drive/MyDrive/W210/Extracting_Text/' + file_name  #Where PDF is Located
Mask_csv_path = '/content/drive/MyDrive/W210/Extracting_Text/' #Where the csv mask output from the DLA is located


In [11]:
#this is to generate the combined df
import pandas as pd
import glob
import os

file_pattern = Mask_csv_path + file_base + '*_mask_mask_summary.csv'
print(file_pattern)

# Use glob to find all files that match the pattern
csv_files = glob.glob(file_pattern)

# Initialize an empty DataFrame to concatenate all the individual DataFrames
combined_df = pd.DataFrame()



# Loop through each file path and read the CSV file into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)

    # Extract specific columns
    coordinates_df = df[['x0f', 'x1f', 'y0f', 'y1f','mask_id','mask_shape','category_lbl']].copy()

    # Extract the page number from the file name
    # Assuming the format is always like '..._page_XXXX_...'
    page_number = os.path.basename(file).split('_')[2]

    # Add a new column for the page number
    coordinates_df['page_number'] = int(page_number)

    # Concatenate the current DataFrame with the combined DataFrame
    combined_df = pd.concat([combined_df, coordinates_df], ignore_index=True)

# Display the combined data
print(combined_df)



/content/drive/MyDrive/W210/Extracting_Text/1603.09631*_mask_mask_summary.csv
       x0f    x1f    y0f    y1f  mask_id  mask_shape category_lbl  page_number
0     47.0  528.0   36.0   55.0        0  (819, 579)      unknown            3
1     61.0  267.0   65.0  270.0        1  (819, 579)         list            3
2     63.0  261.0   68.0  109.0        2  (819, 579)         text            3
3     62.0  264.0  106.0  136.0        3  (819, 579)         text            3
4     62.0  262.0  182.0  211.0        4  (819, 579)         text            3
..     ...    ...    ...    ...      ...         ...          ...          ...
98   294.0  432.0  555.0  573.0       19  (819, 579)        title            2
99   294.0  529.0  571.0  636.0       20  (819, 579)         text            2
100  293.0  425.0  641.0  659.0       21  (819, 579)        title            2
101  294.0  527.0  658.0  744.0       22  (819, 579)         text            2
102  305.0  434.0  757.0  772.0       23  (819, 579) 

In [12]:
#page_data = combined_df[combined_df['page_number'] == 1]
#print(page_data)

In [13]:
import pandas as pd
import fitz  # PyMuPDF
import os
import textwrap


def extract_text_from_scaled_pdf(pdf_path, page_number, coords, new_dimensions):
    """Extract text from specified coordinates in a scaled PDF."""
    doc = fitz.open(pdf_path)
    scaled_doc = fitz.open()  # Create a new empty PDF for scaled pages

    # Scale the specific page
    page = doc.load_page(page_number)
    new_page = scaled_doc.new_page(width=int(new_dimensions[1]), height=int(new_dimensions[0]))
    new_page.show_pdf_page(new_page.rect, doc, page.number)

    # Extract text from the scaled page
    scaled_page = scaled_doc.load_page(0)  # As we have only one page in scaled_doc
    extracted_text = scaled_page.get_text("text", clip=fitz.Rect(coords))
    #extracted_text = ' '.join(extracted_text.split())



    # Clean up
    doc.close()
    scaled_doc.close()
    return extracted_text

# Path to your CSV and PDF files

pdf_file_path = pdf_file_path
# Extract the base name of the PDF file (e.g., '1909.11687' from '1909.11687.pdf')
document_name = os.path.splitext(os.path.basename(pdf_file_path))[0]

# Reading the CSV file
df = combined_df
df_sorted = df.sort_values(by=[df.columns[7], df.columns[4]])
print(df_sorted)

# Initialize an empty string to concatenate text
concatenated_text = ""


# Extract text from each row's coordinates
for index, row in df_sorted.iterrows():
    coords = (row['x0f'], row['y0f'], row['x1f'], row['y1f'])
    page_number = int(row['page_number']) - 1  # Convert to zero-based index for PDF pages
    mask_shape = row['mask_shape']
    category = row['category_lbl']
    numbers = mask_shape.strip("()").split(", ")
    extracted_text = extract_text_from_scaled_pdf(pdf_file_path, page_number, coords, numbers)

    # Split the extracted text into lines
    lines = extracted_text.split('\n')

    # Process each line
    processed_lines = []
    for line in lines:
        # Check if the line ends with a hyphen
        #if line.endswith('-  '):
        #    processed_lines.append(line[:-3])
        #if line.endswith('- '):
            # Remove the hyphen and do not add a space after this line
        #    processed_lines.append(line[:-2])
        if line.endswith('-'):
            processed_lines.append(line[:-1])
        else:
            # Add a space at the end of the line for normal concatenation
            processed_lines.append(line)
    print(f"Extracted Text from Page {page_number+1}, Line {index}:", processed_lines)
    # Join all lines into a single line
    single_line_text = ''.join(processed_lines).strip()
    single_line_text = textwrap.fill(single_line_text, width=80)

    # Format the text for titles
    if category == 'title':
        single_line_text = "\n## " + single_line_text + " ##\n"

    concatenated_text += "\n" + single_line_text


# Print the concatenated text
print("\nConcatenated Text:\n", concatenated_text)

# Save the concatenated text into a .txt file
output_file_path = f'/content/drive/MyDrive/W210/Extracting_Text/{document_name}_extracted_text.txt'
with open(output_file_path, 'w') as file:
    file.write(concatenated_text)

print(f"Concatenated text saved to {output_file_path}")



      x0f    x1f    y0f    y1f  mask_id  mask_shape category_lbl  page_number
32  108.0  472.0   80.0  105.0        0  (819, 579)        title            1
33  206.0  372.0  110.0  129.0        1  (819, 579)         text            1
34  158.0  419.0  122.0  176.0        2  (819, 579)         text            1
35  266.0  309.0  185.0  201.0        3  (819, 579)        title            1
36   48.0  529.0  198.0  255.0        4  (819, 579)         text            1
..    ...    ...    ...    ...      ...         ...          ...          ...
78  293.0  530.0  682.0  718.0       23  (819, 579)         text            4
51   47.0  528.0   36.0   55.0        0  (819, 579)      unknown            5
52   46.0  286.0   64.0  282.0        1  (819, 579)         text            5
53   48.0  280.0  173.0  197.0        2  (819, 579)         text            5
54   48.0  282.0  195.0  212.0        3  (819, 579)         text            5

[103 rows x 8 columns]
Extracted Text from Page 1, Line 32: ['D

In [15]:
#Json Creation
import json
import fitz  # PyMuPDF

def extract_text_from_scaled_pdf(pdf_path, page_number, coords, new_dimensions):
    """Extract text from specified coordinates in a scaled PDF."""
    doc = fitz.open(pdf_path)
    scaled_doc = fitz.open()  # Create a new empty PDF for scaled pages

    # Scale the specific page
    page = doc.load_page(page_number)
    new_page = scaled_doc.new_page(width=int(new_dimensions[1]), height=int(new_dimensions[0]))
    new_page.show_pdf_page(new_page.rect, doc, page.number)

    # Extract text from the scaled page
    scaled_page = scaled_doc.load_page(0)  # As we have only one page in scaled_doc
    extracted_text = scaled_page.get_text("text", clip=fitz.Rect(coords))

    # Clean up
    doc.close()
    scaled_doc.close()
    return extracted_text

# Path to your PDF file
pdf_file_path = pdf_file_path
document_name = os.path.splitext(os.path.basename(pdf_file_path))[0]

# Reading the CSV file
# Make sure you define 'combined_df' before this line
df = combined_df
df_sorted = df.sort_values(by=[df.columns[7], df.columns[4]])

# Initialize the JSON structure outside the loop
json_structure = {
    "paper_id": document_name,
    "title": "",  # Set this based on your data/logic
    "paper_text": []
}



# Extract and process text
for index, row in df_sorted.iterrows():
    coords = (row['x0f'], row['y0f'], row['x1f'], row['y1f'])
    page_number = int(row['page_number']) - 1
    mask_shape = row['mask_shape']
    category = row['category_lbl']
    numbers = mask_shape.strip("()").split(", ")
    extracted_text = extract_text_from_scaled_pdf(pdf_file_path, page_number, coords, numbers)

    # Process the extracted text
    lines = extracted_text.split('\n')
    processed_lines = [line[:-1] if line.endswith('-') else line for line in lines]
    single_line_text = ''.join(processed_lines).strip()
    single_line_text = textwrap.fill(single_line_text, width=80)

    # Add to JSON structure
    section_dict = {
        "section_name": "",  # Set this based on your data/logic
        "section_text": single_line_text,
        "section_annotation": category,
        "section_page": page_number + 1,
        "section_column": 0,  # Set this based on your data/logic
        "section_location": [coords]
    }
    json_structure["paper_text"].append(section_dict)

# Convert the structured dictionary to JSON
json_output = json.dumps(json_structure, indent=4)

# Save the JSON output to a file
json_output_file_path = f'/content/drive/MyDrive/W210/Extracting_Text/{document_name}_extracted_text.json'
with open(json_output_file_path, 'w') as json_file:
    json_file.write(json_output)

print(f"JSON output saved to {json_output_file_path}")

JSON output saved to /content/drive/MyDrive/W210/Extracting_Text/1603.09631_extracted_text.json
